In [39]:
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"

import sys
import os

module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 

sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [48]:
from APIutils import *
import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
from datetime import date, datetime 
import datetime
from pathlib import Path 
import os 
import glob
import nltk
from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
import pandas as pd
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

from utils import *
import random
from timeit import default_timer as timer
import time
import datetime


#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
TRANSFORMERMODEL = 'bert-base-nli-mean-tokens'
from dynamoDButils import *
from APIutils import *
from seedterms import *


def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


Cell imports done @ 2023-02-13 14:00:01


In [237]:
def restrictiveScore(df):
    TECHLABEL='technology'
    INNOLABEL='innovation'
    BUSILABEL='business innovation'
    SCIXLABEL= 'science'
    FEDLABEL = 'federal'
    THRESHOLD = 0.4

    cumulative = 0.0
    topic1 = df.iloc[0] ['label']
    topic2 = df.iloc[1] ['label']
    score1 = round(df.iloc[0] ['score'],2)
    score2 = round(df.iloc[1] ['score'],2)
    cumulative = score1 + score2
    ### How does the article score
    if (((topic1 == TECHLABEL and topic2 == INNOLABEL) or
         (topic1 == TECHLABEL and topic2 == BUSILABEL) or
         (topic1 == TECHLABEL and topic2 == SCIXLABEL) or 
         (topic1 == BUSILABEL and topic2 == BUSILABEL) or 
         (topic1 == INNOLABEL and topic2 == TECHLABEL) or 
         (topic1 == SCIXLABEL and topic2 == TECHLABEL)) and 
        (cumulative >= THRESHOLD)
    ):

        return (True, topic1, topic2, cumulative)
    else:
        return (False, topic1, topic2, cumulative)

def format_results(classify):
    labels = classify['labels']
    scores = classify['scores']
    scores_csv = []
    for l in enumerate(classify['labels']):
        # note labels are in the form (0, 'public'), so can just use the oth term in l as index to scores
        scores_csv.append(
            {
                "label" : l[1],
                "score" : scores[l[0]]  
            }
        )
    return pd.DataFrame(scores_csv)


In [343]:
# Create empty dict to add the new scores to 
test_dict = {
    "url":[], "title":[], "oldScore":[], "newScore":[], "topic1":[], "topic2":[], "weightScore":[]
}

### Experiment with Results From our production Scoring System

Cobbled from our EC2 Lambda code.

In [52]:
%%time

RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

os.environ['AWS_DEFAULT_REGION']= region
print (f"env : {os.getenv('AWS_DEFAULT_REGION')}")
print('\n\nconnecting ...')
exists = table_exists(RESOURCE, REGION, TABLE)
complete(state= "DB read")

env : us-east-1


connecting ...
table dynamodb.Table(name='Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev') status ACTIVE
Table dynamodb.Table(name='Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev') stats: Creation Time : 2022-09-13 02:11:28.285000+00:00

Cell DB read @ 2023-02-13 14:00:23
CPU times: user 157 ms, sys: 0 ns, total: 157 ms
Wall time: 803 ms


In [53]:
resp = extendedScanArticlesWithKey(RESOURCE,TABLE, columns="title,summary", keyword="url")

page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 4
extendedScanArticles: Length: 3948 


In [54]:
# Serializing json
indx = 0
jsndict = {}
for item in resp:
    jsndict[str(indx)]= {
        "url" : item['url']['S'],
        "title" : item['title']['S'],
        "summary": item['summary']['S']
    }
    indx += 1
    
json_object = json.dumps(jsndict, indent=4)
jsonfile = join(JSONPath, "semanticSimArticles.json")


with open(jsonfile, "w") as outfile:
     outfile.write(json_object)

In [55]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [725]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
xposedf = df.transpose() # pivot the matrix

jsnf='semanticSimArticles.json' 


In [726]:
article_ls = xposedf['title'].values.tolist()


In [727]:
# Widget to select articles in the newsfeed repository for re-evaluation 

articleWD = widgets.Select(
    options=article_ls,
    value=article_ls[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='70%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

articleWD.observe(on_change)
display(articleWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='70%'), options=("Analysis: There's a lar…

In [749]:
val = articleWD.value
val

'What Are The Top OceanSound Government Contracts?'

In [750]:
url = xposedf.url[xposedf.title == val].unique()[0] 
print(url)

https://executivegov.com/articles/what-are-the-top-oceansound-government-contracts


In [751]:
# # extract the summary from a candidate URL
# # url = "https://www.engadget.com/the-lastpass-hack-was-worse-than-the-company-first-reported-000501559.html"
# # url = "https://www.msn.com/en-us/news/world/russia-pulls-troops-from-front-lines-after-soldiers-beaten-by-own-allies/ar-AA17gkxJ"
# # url = "https://www.msn.com/en-us/news/world/ground-breaking-dna-sequencing-study-shows-vikings-weren-t-who-we-believed-them-to-be/ss-AA16bjqo"
# # url = "https://www.msn.com/en-us/news/world/putin-ally-says-u-s-will-become-strait-named-after-stalin-for-ukraine-war/ar-AA17gdO5"
# # url = "https://www.cnbc.com/2023/01/09/uk-pm-sunak-restarts-talks-with-softbank-about-listing-arm-in-london.html"
# # url = "https://www.nextgov.com/emerging-tech/2023/01/dhs-awards-11m-federal-labs-contracts-seeking-groundbreaking-tech/381881/"
# # url = "https://ew.com/music/burt-bacharach-dead-legendary-composer-was-94/"
# # url = "https://www.cnn.com/2023/01/31/health/mpox-public-health-emergency-lessons"
# # url = "https://www.businessinsider.com/apple-executives-violated-worker-rights-tim-cook-labor-officials-nlrb-2023-1"
# url = "https://www.engadget.com/eu-investigating-broadcoms-61-billion-deal-buy-v-mware-234418707.html"
# url = "https://www.businessinsider.com/google-microsoft-ai-search-war-will-add-to-carbon-emissions-2023-2"
results = extractArticle(site_url=url)
complete(state="extraction done")


Cell extraction done @ 2023-02-13 20:02:08


In [752]:
THRESHOLD = 0.8

summary = results['summary']
candidate_labels = [ 'technology', 'innovation','politics', 'electronics'
                    ,'entertainment', 'economics', 'advertisement', 'social'
                    ,'games', 'sports', 'health', 'science', 'business innovation'
                    ,'federal', 'public', 'government', 'data', 'ai'
                    , 'application','security', 'cyber security', 'networks'
                    , 'chatgpt', 'openai', 'open ai' ]
classify = classifier( summary
                              , candidate_labels
                              ,device=0     
                     )

In [753]:
score = round (
    (   round (classify['scores'][classify['labels'].index('technology')], 2 )
           + round (classify['scores'][classify['labels'].index('innovation')], 2 ) 
           + round (classify['scores'][classify['labels'].index('politics')], 2 )
             + round (classify['scores'][classify['labels'].index('electronics')], 2 )
           + round (classify['scores'][classify['labels'].index('federal')], 2 )
           + round (classify['scores'][classify['labels'].index('public')], 2 )
           + round (classify['scores'][classify['labels'].index('government')], 2 )
           + round (classify['scores'][classify['labels'].index('data')], 2 )
           + round (classify['scores'][classify['labels'].index('ai')], 2 )
           + round (classify['scores'][classify['labels'].index('application')], 2 )
           + round (classify['scores'][classify['labels'].index('security')], 2 )
             + round (classify['scores'][classify['labels'].index('cyber security')], 2 )
           + round (classify['scores'][classify['labels'].index('chatgpt')], 2 )
           + round (classify['scores'][classify['labels'].index('openai')], 2 )
           + round (classify['scores'][classify['labels'].index('business innovation')], 2 )
           + round (classify['scores'][classify['labels'].index('open ai')], 2 )
        )
    ,2)
# print (f"SCORE: {score}")
if score >= THRESHOLD: 
    print(f"ACCEPTED THIS ARTICLE. SCORE: {score}")
else:
    print(f"REJECTED THIS ARTICLE. SCORE: {score}")
    
print(f"\t\ttechnology: {round (classify['scores'][classify['labels'].index('technology')], 2 )}")
print(f"\t\tinnovation: {round (classify['scores'][classify['labels'].index('innovation')], 2 )}")
print(f"\t\tpolitics: {round (classify['scores'][classify['labels'].index('politics')], 2 )}")
print(f"\t\telectronics: {round (classify['scores'][classify['labels'].index('electronics')], 2 )}")
print(f"\t\tfederal: {round (classify['scores'][classify['labels'].index('federal')], 2 )}")
print(f"\t\tpublic: {round (classify['scores'][classify['labels'].index('public')], 2 )}")
print(f"\t\tgovernment: {round (classify['scores'][classify['labels'].index('government')], 2 )}")
print(f"\t\tdata: {round (classify['scores'][classify['labels'].index('data')], 2 )}")
print(f"\t\tai: {round (classify['scores'][classify['labels'].index('ai')], 2 )}")
print(f"\t\tapplication: {round (classify['scores'][classify['labels'].index('application')], 2 )}")
print(f"\t\tsecurity: {round (classify['scores'][classify['labels'].index('security')], 2 )}")
print(f"\t\tcyber security: {round (classify['scores'][classify['labels'].index('cyber security')], 2 )}")
print(f"\t\tchatgpt: {round (classify['scores'][classify['labels'].index('chatgpt')], 2 )}")
print(f"\t\topenai: {round (classify['scores'][classify['labels'].index('openai')], 2 )}")
print(f"\t\topenai: {round (classify['scores'][classify['labels'].index('business innovation')], 2 )}")
print(f"\t\topen ai: {round (classify['scores'][classify['labels'].index('open ai')], 2 )}")

complete(state="scoring")

REJECTED THIS ARTICLE. SCORE: 0.76
		technology: 0.05
		innovation: 0.04
		politics: 0.02
		electronics: 0.01
		federal: 0.02
		public: 0.15
		government: 0.02
		data: 0.03
		ai: 0.07
		application: 0.04
		security: 0.02
		cyber security: 0.02
		chatgpt: 0.07
		openai: 0.04
		openai: 0.04
		open ai: 0.12

Cell scoring @ 2023-02-13 20:03:29


In [764]:
classify

{'sequence': 'DMI, a division of OceanSound Partners, offers services for intelligent digital transformation at the enterprise level, aiming to optimize and expand existing IT infrastructure.\nThe Surface Deployment and Distribution Command of the Army will benefit from the contract’s provision of engineering services and analysis, training materials, tools, traffic engineering publications, and traffic studies.\nOceanSound Partners headquarters operates at 320 Park Avenue, 8th Floor, New York, New York.\nOceanSound Partners’ companies have revenue of $2.3 billion with over 9,000 experts.\nHere are the OceanSound Partners and what they do.',
 'labels': ['public',
  'open ai',
  'social',
  'chatgpt',
  'ai',
  'technology',
  'economics',
  'business innovation',
  'openai',
  'application',
  'innovation',
  'data',
  'entertainment',
  'science',
  'advertisement',
  'security',
  'government',
  'health',
  'federal',
  'cyber security',
  'politics',
  'electronics',
  'networks',


Experiment  Using a more restictive set of classifiers?

In [754]:
df = pd.DataFrame()
df = format_results (classify)
# print (f"unsorted :\n\n {df}")

df = df.sort_values('score', ascending=False)
print (f"sorted :\n\n {df}")


sorted :

                   label     score
0                public  0.148189
1               open ai  0.118271
2                social  0.070437
3               chatgpt  0.069689
4                    ai  0.067957
5            technology  0.048557
6             economics  0.046769
7   business innovation  0.043880
8                openai  0.042741
9           application  0.040090
10           innovation  0.036260
11                 data  0.032197
12        entertainment  0.027417
13              science  0.025853
14        advertisement  0.023990
15             security  0.021211
16           government  0.020083
17               health  0.017377
18              federal  0.017337
19       cyber security  0.017141
20             politics  0.015888
21          electronics  0.014740
22             networks  0.013296
23                games  0.010394
24               sports  0.010237


In [755]:
rtn = restrictiveScore(df)
print (rtn)
if rtn[0]:
    print (f"Article accepted: Topic1 : {rtn[1]},Topic2 : {rtn[2]} :: Score : {rtn[3]}")
else:
    print (f"Article fails: Topic1 : {rtn[1]},Topic2 : {rtn[2]} :: Score : {rtn[3]}")


(False, 'public', 'open ai', 0.27)
Article fails: Topic1 : public,Topic2 : open ai :: Score : 0.27


In [756]:
CUTOFF = 0.45
# Weighted topic ranking scoring system 

weight_score = round ((
    (classify['scores'][classify['labels'].index('technology')] * 0.8) +
    (classify['scores'][classify['labels'].index('innovation')] * 0.8) +
    (classify['scores'][classify['labels'].index('ai')] * 0.7) +
    (classify['scores'][classify['labels'].index('openai')] * 0.7) +
    (classify['scores'][classify['labels'].index('open ai')] * 0.7) +
    (classify['scores'][classify['labels'].index('data')] * 0.6) +
    (classify['scores'][classify['labels'].index('business innovation')] * 0.6) +
    (classify['scores'][classify['labels'].index('science')] * 0.6) +
    (classify['scores'][classify['labels'].index('federal')] * 0.6) +
    (classify['scores'][classify['labels'].index('cyber security')] * 0.5) +
    (classify['scores'][classify['labels'].index('chatgpt')] * 0.5) +
    (classify['scores'][classify['labels'].index('government')] * 0.5) +
    (classify['scores'][classify['labels'].index('security')] * 0.4) +
    (classify['scores'][classify['labels'].index('public')] * 0.4) +
    (classify['scores'][classify['labels'].index('electronics')] * 0.3) +
    (classify['scores'][classify['labels'].index('networks')] * 0.3) +
    (classify['scores'][classify['labels'].index('application')] * 0.3) +
    
    (classify['scores'][classify['labels'].index('politics')] * -0.5) +
    (classify['scores'][classify['labels'].index('social')] * -0.7) +
    (classify['scores'][classify['labels'].index('games')] * -0.8) +
    (classify['scores'][classify['labels'].index('entertainment')] * -0.8) +
    (classify['scores'][classify['labels'].index('sports')] * -0.8) +
    (classify['scores'][classify['labels'].index('advertisement')] * -1.0) 
    ), 2)


if weight_score >= CUTOFF: 
    print(f"ACCEPTED THIS ARTICLE. SCORE: {weight_score}")
else:
    print(f"REJECTED THIS ARTICLE. SCORE: {weight_score}")
    

REJECTED THIS ARTICLE. SCORE: 0.32


In [757]:
# Add the re-evalaution scores dictionary
# add scores and article information to dictionary
test_dict['url'].append(url)
test_dict['title'].append(val)
test_dict['oldScore'].append(score)
test_dict['newScore'].append(rtn[3])
test_dict['topic1'].append(rtn[1])
test_dict['topic2'].append(rtn[2])
test_dict['weightScore'].append(weight_score)

In [758]:
scoresdf = pd.DataFrame(test_dict)
scoresdf.tail()
# test_dict

,url,title,oldScore,newScore,topic1,topic2,weightScore
32,https://www.govexec.com/technology/2023/02/con...,Congress Told HHS to Set Up a Health Data Netw...,0.73,0.36,public,health,0.32
33,https://www.businessinsider.com/google-microso...,The internet contributes 1.6 billion annual to...,0.84,0.34,technology,open ai,0.48
34,https://governmentciomedia.com/va-outlines-nur...,VA Outlines Nursing Recruitment Strategy Amid ...,0.77,0.24,technology,health,0.41
35,https://markets.businessinsider.com/news/stock...,Chegg plunges 23% as it shrugs off the threat ...,0.86,0.40,technology,public,0.46
36,https://executivegov.com/articles/what-are-the...,What Are The Top OceanSound Government Contracts?,0.76,0.27,public,open ai,0.32


In [759]:
jsonfile = join(JSONPath, "articleScoringExperiment.json")
print(jsonfile)

scoresdf.to_json(jsonfile)
complete(state="updates written: ")

/home/deleidos/Notebooks/JSONData/articleScoringExperiment.json

Cell updates written:  @ 2023-02-13 20:03:40


### Anna: You can stop here or continue

Continuing might be using the same URL to score via the other methods, such as:

##### Calculating Similarity

The inspiration for this approach originates from a reading of this article:

###### BERT For Measuring Text Similarity
[High-performance semantic similarity with BERT](https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1)

Our approach will be to read all the summaries from the articles data base, use them to encode and create embeddings and compare the candidate articles to them, then observe scores.

In [9]:
%%time
# Anna , if you want try different transformers?
model = SentenceTransformer(TRANSFORMERMODEL)
complete(state='model initialised')


Cell model initialised @ 2023-02-09 22:56:25
CPU times: user 1.07 s, sys: 160 ms, total: 1.23 s
Wall time: 1.17 s


Next choose the corpus of news articles 

In [10]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsn2WD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsn2WD.observe(on_change)
display(jsn2WD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [11]:
# iWD.value
jsnf = jsn2WD.value
print (f"{jsnf=} ")
# probable source jsnf='semanticSimArticles.json'  
# jsnfile = osp.join (JSONPath,jsnf)
# xposedf = pd.read_json(jsnfile)
# xposedf = xposedf.transpose() # pivot the matrix
# xposedf

jsnf='semanticSimArticles.json' 


#### Read Corpus  articles and create sentence embeddings
Here, we begin the process of looking at a collection of articles and rating them for inclusion vice a model constructed from the existing corpus of articles

In [12]:
jsnfile = osp.join (JSONPath,jsnf)
cSummaries = pd.read_json(jsnfile)
cSummaries = cSummaries.transpose() # pivot the matrix
cSummaries

,url,title,summary
0,https://www.cnn.com/2023/01/16/politics/overcl...,Analysis: There's a larger 'classified' proble...,A version of this story appeared in CNN’s What...
1,https://www.techrepublic.com/article/best-digi...,Best digital collaboration tools for your busi...,These 10 real-time digital collaboration apps ...
2,https://www.businessinsider.com/i-asked-chatgp...,I asked ChatGPT to do my work and write an Ins...,"I asked ChatGPT, a new AI chatbot from OpenAI,..."
3,https://www.engadget.com/netflixs-dog-and-boy-...,Netflix's 'Dog and Boy' anime causes outrage f...,"In 2016, Studio Ghibli co-founder and director..."
4,https://www.macworld.com/article/1484137/apple...,Apple’s next battle in its ‘silent war’ agains...,While we’re all waiting for Apple to unveil it...
...,...,...,...
3539,https://www.cnn.com/2023/01/10/asia/china-fune...,China: Satellite images capture crowding at cr...,CNN —Satellite images taken over a number of C...
3540,https://www.techrepublic.com/article/nrf-2023-...,NRF 2023: How ambient computing could change r...,Wiliot’s cloud-connected tags attempt to solve...
3541,https://www.macworld.com/article/1476747/homep...,"Defying logic, Apple announces 2nd-gen HomePod...",A day after unveiling updated Macs with M2 pro...
3542,https://www.digitaltrends.com/dtdeals/lenovo-l...,Lenovo Legion gaming PCs and laptops are on sa...,"As we’re moving through January, we keep seein..."


In [13]:
cSentences = []
cSentences = cSummaries.summary.to_list()

complete(state="models")
cSentences[0]


Cell models @ 2023-02-09 22:56:51


'A version of this story appeared in CNN’s What Matters newsletter. To get it in your inbox, sign up for free here.CNN —This is getting absurd.Americans have had the opportunity to choose from between three people to be president since 2016, and all three have been found to have classified documents they should not have.Clearly, there’s a larger problem.Here’s what we don’t know:We, the public, don’t know what documents President Joe Biden had locked up in his garage or in the manila folder in his Penn Biden Center office.We don’t know specifics about what documents the FBI photographed on the floor at former President Donald Trump’s Mar a Lago resort.We don’t even know for sure what documents were on Hillary Clinton’s email server, although we do know that dozens of those documents were retroactively classified years after the fact, long after the existence of her private email server became a political albatross.Here’s what we do know:Classified documents, and their appearance in ema

#### Candidates for inclusion
Next read in the candidate collection and use it to create an embedding

In [14]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [15]:
jsnf = jsnWD.value
print (f"{jsnf=} ")
# probable source : jsnf='semanticSimArticlesTestingDF.json'
jsnfile = osp.join (JSONPath,jsnf)
xSummaries = pd.read_json(jsnfile)
# xSummaries = xSummaries.transpose() # pivot the matrix
xSummaries

jsnf='semanticSimArticlesTestingDF.json' 


,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/news/spacex-launch-tod...,spacex falcon rocket carrying gps navigation s...,0.8183,0.4993,0.4775,0.4735,0.4719,0.480550,0.531412,0.511708,0.454913,0.436846,0.434590,0.473894,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/video/chatgpt-jobs-art...,rise could chatgpt take job chatbots artificia...,0.7697,0.7255,0.7166,0.7103,0.6910,0.710850,0.637201,0.625964,0.625526,0.619846,0.619757,0.622773,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/news/ukraine-military-...,jonas ohman founder ceo blue yellow lithuania ...,0.4923,0.4892,0.4892,0.4847,0.4596,0.488850,0.457436,0.457436,0.430608,0.427529,0.421351,0.438872,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/news/metaverse-making-...,metaverse asset digital art video game charact...,0.6600,0.6304,0.6264,0.6229,0.6044,0.626567,0.614686,0.609527,0.597931,0.587996,0.571264,0.596281,games,0.463839,entertainment,0.240798


In [16]:
xSentences = preProcessing(xSummaries)


Cell models @ 2023-02-09 22:57:11


In [17]:
print (f"xSentences[0] : {xSentences[0]}")

xSentences[0] : economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look


### Encode the sentences:

In [18]:
xSentence_embeddings = model.encode(xSentences)
# xSentence_embeddings.shape

In [19]:
cSentence_embeddings = model.encode(cSentences)
# cSentence_embeddings.shape

In [20]:
print (f"cSentence_embeddings: {cSentence_embeddings[0]}", end="\n\n")
print (f"xSentence_embeddings: {xSentence_embeddings[0]}")

cSentence_embeddings: [-1.05547398e-01  6.77061677e-01 -6.27748221e-02 -1.53308123e-01
  2.28242964e-01 -8.57265592e-01  8.89505506e-01 -1.51370302e-01
  8.47467631e-02  1.16134301e-01 -2.34682813e-01  1.69230849e-01
  2.10549787e-01  5.94752610e-01 -8.55719805e-01  5.07248938e-01
  4.93643992e-03  1.75873905e-01  2.74112165e-01  7.01239407e-02
 -3.91417503e-01 -4.09466803e-01  1.98670536e-01 -1.30761266e-01
  1.24996388e+00  7.56384075e-01 -1.31943271e-01 -2.66239673e-01
 -8.48192930e-01  2.49533206e-01 -3.41379702e-01 -1.67302787e-01
 -5.78156948e-01  4.34515178e-02 -2.19298452e-01  5.90164065e-01
  2.87288487e-01 -3.14747214e-01 -2.79607832e-01  3.04193586e-01
 -1.34115472e-01 -2.04753160e-01  2.26539701e-01  2.85508275e-01
 -5.50704598e-01 -4.39586371e-01  7.41265416e-01  9.46080983e-01
  2.78816223e-01 -8.28725159e-01  7.57273555e-01  1.65659666e-01
  4.60772455e-01  1.96345031e-01  1.95734888e-01  1.43017218e-01
 -2.82591656e-02 -1.28287458e+00 -5.91428459e-01 -4.83238041e-01
 -3

In [21]:
def getHiFive(cs):
    similarities = []
    ord         = 0
    for elt in cs:
        ord += 1
        similarities.append([ord, elt])
    s = sorted(similarities,key=lambda x: x[1] )
    s = s[len(s)-6:len(s)-1]
    print (f"S: {s}")
    return s

In [22]:
scores = []
for i in range (0, len(xSentence_embeddings)):
    cs = cosine_similarity(
        [xSentence_embeddings[i]],
        cSentence_embeddings[:]
    )
    s = getHiFive(cs[0])
    scores.append(s)
complete(state="xSentence_embeddings")

S: [[816, 0.7517196], [1087, 0.7526125], [1748, 0.75273156], [338, 0.75290674], [2771, 0.7551762]]
S: [[1453, 0.8327645], [2367, 0.8346276], [3425, 0.83492464], [3321, 0.842566], [91, 0.8530389]]
S: [[1827, 0.70135057], [2364, 0.7028035], [2605, 0.7057362], [444, 0.7084293], [528, 0.7085992]]
S: [[3436, 0.75489545], [1271, 0.75721323], [1577, 0.75871056], [1861, 0.76303065], [316, 0.77057683]]
S: [[285, 0.7746706], [1514, 0.7797184], [2437, 0.7800753], [2209, 0.7905019], [1455, 0.7926855]]
S: [[1803, 0.8080007], [2998, 0.8115827], [109, 0.8118024], [2160, 0.81862795], [432, 0.8206656]]
S: [[729, 0.7727531], [3469, 0.77411973], [2668, 0.7742644], [1076, 0.7785818], [928, 0.78920674]]
S: [[1178, 0.57486856], [1906, 0.582057], [2248, 0.58622456], [2838, 0.5951249], [1183, 0.6228925]]
S: [[469, 0.7197938], [423, 0.72295403], [1163, 0.727957], [250, 0.7331795], [2156, 0.76915103]]
S: [[1397, 0.75571156], [1291, 0.75907075], [3509, 0.75907075], [1469, 0.7643063], [1215, 0.76462233]]
S: [[216

S: [[554, 0.75096303], [2540, 0.7548727], [736, 0.75533485], [3469, 0.758388], [1580, 0.7600156]]
S: [[3008, 0.7304355], [3476, 0.73054004], [756, 0.73214114], [1009, 0.7345829], [63, 0.73489416]]
S: [[3008, 0.7304355], [3476, 0.73054004], [756, 0.73214114], [1009, 0.7345829], [63, 0.73489416]]
S: [[1178, 0.57486856], [1906, 0.582057], [2248, 0.58622456], [2838, 0.5951249], [1183, 0.6228925]]
S: [[1078, 0.84852564], [1612, 0.8489377], [1623, 0.8516233], [396, 0.85897684], [1243, 0.8705381]]
S: [[2497, 0.7965908], [3365, 0.7978698], [900, 0.8021819], [1569, 0.80841964], [2662, 0.8086312]]
S: [[2435, 0.79654443], [2171, 0.7969128], [3139, 0.8039552], [1381, 0.8139938], [197, 0.83019304]]
S: [[2254, 0.8082277], [586, 0.8145005], [3002, 0.8170264], [1881, 0.8356285], [2648, 0.83719254]]
S: [[862, 0.8788201], [2483, 0.87884307], [1690, 0.8817648], [2883, 0.88312113], [2626, 0.8837119]]
S: [[1586, 0.82548904], [58, 0.8266156], [2806, 0.8291862], [1940, 0.838122], [2831, 0.8391906]]
S: [[2152

In [23]:
# Try for a silly comparison just to prove everything works as we might hope.
# generated from https://randomwordgenerator.com/sentence.php
sillysentences = [
     "I know many children ask for a pony, but I wanted a bicycle with rockets strapped to it."
    ,"The swirled lollipop had issues with the pop rock candy."
    ,"As the years pass by, we all know owners look more and more like their dogs."
    ,"When nobody is around, the trees gossip about the people who have walked under them."
    ,"She saw the brake lights, but not in time."
    ,"Karen realized the only way she was getting into heaven was to cheat."
    ,"""
The EU is investigating Broadcom\'s $61 billion deal to buy VMware One of biggest (and dullest) 
big tech acquisitions ever may be blocked by the European Commission.
The European Union plans to carry out a full-scale investigation of Broadcom\'s $61 billion bid to buy VMware.
The European Commission warns those actions "could lead to higher prices, lower quality and less innovation 
for business customers, and ultimately consumers."\nNot now Turned on Turn onAs one of the larger tech 
acquisitions of 2022, Broadcom\'s bid to buy VMware was certain to draw scrutiny.\nIf the deal were to 
fall through, it would be a bitter repeat of Broadcom\'s 2018 attempt to buy chipmaker Qualcomm.
"""
]

sillysentence_embeddings = model.encode(sillysentences)
# sillysentence_embeddings.shape

In [24]:
scores = []
for i in range (0, len(sillysentence_embeddings)):
    cs = cosine_similarity(
        [sillysentence_embeddings[i]],
        cSentence_embeddings[:]
        )
    s = getHiFive(cs[0])
    scores.append(s)
    
complete(state="silly estimations")    

S: [[3072, 0.5249635], [1794, 0.53396606], [3035, 0.53396606], [1204, 0.5502929], [3241, 0.5532384]]
S: [[2614, 0.5675092], [660, 0.5787128], [3438, 0.58921546], [137, 0.5927881], [1032, 0.6092144]]
S: [[3464, 0.57091963], [2199, 0.5799371], [2557, 0.5818205], [701, 0.598508], [2665, 0.6000067]]
S: [[500, 0.5535485], [1122, 0.5651385], [2574, 0.5833842], [3155, 0.5851338], [2277, 0.5914216]]
S: [[3427, 0.5133487], [9, 0.5158811], [1032, 0.53304183], [2781, 0.53431463], [1005, 0.53517616]]
S: [[2691, 0.561372], [661, 0.5618552], [2539, 0.56266516], [2277, 0.56275076], [1413, 0.5793836]]
S: [[1728, 0.8715508], [2808, 0.8909322], [1390, 0.89970034], [1120, 0.9145456], [617, 0.9238434]]

Cell silly estimations @ 2023-02-09 22:58:35


In [25]:
# Reverse the list
print (f"Total Scores : {len(scores)}")
for score in scores:
    score = score[::-1]
    print(f"Score:{score}")

Total Scores : 7
Score:[[3241, 0.5532384], [1204, 0.5502929], [3035, 0.53396606], [1794, 0.53396606], [3072, 0.5249635]]
Score:[[1032, 0.6092144], [137, 0.5927881], [3438, 0.58921546], [660, 0.5787128], [2614, 0.5675092]]
Score:[[2665, 0.6000067], [701, 0.598508], [2557, 0.5818205], [2199, 0.5799371], [3464, 0.57091963]]
Score:[[2277, 0.5914216], [3155, 0.5851338], [2574, 0.5833842], [1122, 0.5651385], [500, 0.5535485]]
Score:[[1005, 0.53517616], [2781, 0.53431463], [1032, 0.53304183], [9, 0.5158811], [3427, 0.5133487]]
Score:[[1413, 0.5793836], [2277, 0.56275076], [2539, 0.56266516], [661, 0.5618552], [2691, 0.561372]]
Score:[[617, 0.9238434], [1120, 0.9145456], [1390, 0.89970034], [2808, 0.8909322], [1728, 0.8715508]]


### Test Results
From the tests above, it seems clear that the estimation method works. One question is where we do the cutoffs?

Results in the 0.0 to 0.65x range seem to be below cutoff.

Results in the 0.7 and above range seem to be right for inclusion,

But what about something in the range 0.6 to 0.699?

In [35]:
cSummaries.iloc[1728]['summary']# write out the KeyBERT Scores; 

'Network disaggregation has become a hot topic within the telco industry. But what exactly is it? In a nutshell, disaggregation is simply the decoupling of network infrastructure, enabling telcos to combine hardware and software components from different vendors to build their own customized network.There are different types of network disaggregation: management plane disaggregation, system disaggregation, services plane disaggregation, silicon disaggregation and control plane disaggregation. But what unites them is that they’re low cost, flexible, and involve no vendor-lock in. So, it’s no wonder that network disaggregation is gaining momentum.Why Do We Need Disaggregation?Almost three years on from COVID and the effects of the pandemic are still being felt by the telco sector. Whilst working from home has enabled many of us to benefit from lower commuting costs and a better work/life balance, the increased demand and strain on networks hasn’t been so comfortable for telcos! Increased

In [27]:
xSummaries.head()

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970


In [ ]:
scores[0]

In [ ]:
cArticleURLs = cSummaries.url.to_list()
# print ( f"scores[0]: {scores[0]}")
for score in scores[0]:
    articleID = score[0]
    print (f"Score: {articleID} URL most similar: {cArticleURLs[articleID]}")

In [ ]:
print (scores[0][0][0])
print (f" before: {xSummaries.iloc[0]}")
xSummaries.iloc[0]['KB1'] = scores[0][0][0]
DF2 = DF2 = pd.DataFrame([[11,12,13,14]], columns = ('x', 'y', 'z', 'u'), index=['A']) 
xSummaries.iloc[0][['KB1', 'KB2', 'KB3', 'KB4'] ] = DF2[['x', 'y', 'z', 'u']]





In [28]:
cSummaries.iloc[617]['summary']

'Kate Howells found out she had been laid off from Google hours after giving birth.She was one of the 12,000 Google employees caught in the latest slew of tech layoffs.Howells said Google has not responded to her query about whether she will get her maternity leave benefits.Sign up for our newsletter for the latest tech news and scoops — delivered daily to your inbox. Loading Something is loading. Thanks for signing up! Access your favorite topics in a personalized feed while you\'re on the go. download the app Email address By clicking ‘Sign up’, you agree to receive marketing emails from Insider as well as other partner offers and accept our Terms of Service and Privacy PolicyKate Howells gave birth to her second child on January 19. She said that barely ten hours later, while still in the hospital holding her baby, she found out that she had been laid off from her job at Google.She was one of the 12,000 Google employees caught in the latest slew of tech layoffs, which was announced 

In [ ]:
cSummaries.iloc[12]['summary']

In [ ]:
score